In [16]:
import pickle
import time
from tqdm import tqdm
from bitmex import bitmex
from datetime import timedelta, datetime
from dateutil import parser
import math
import requests
import sys
import pandas as pd
import json

In [17]:
products_response = requests.get("https://api.pro.coinbase.com/products")

In [55]:
norm_products = pd.json_normalize(json.loads(products_response.text))

In [56]:
print(norm_products)

            id base_currency quote_currency base_min_size base_max_size  \
0      ENJ-BTC           ENJ            BTC          0.32         28000   
1      VGX-USD           VGX            USD          0.23         13000   
2     ALGO-GBP          ALGO            GBP             1        160000   
3     USDT-GBP          USDT            GBP          0.97        320000   
4      NKN-BTC           NKN            BTC           1.7        120000   
..         ...           ...            ...           ...           ...   
399    OMG-USD           OMG            USD           0.1         54000   
400  DOGE-USDT          DOGE           USDT           3.8        890000   
401  MATIC-EUR         MATIC            EUR           0.5        280000   
402   XYO-USDT           XYO           USDT            17        880000   
403   AAVE-USD          AAVE            USD         0.003          2500   

    quote_increment base_increment display_name min_market_funds  \
0         0.0000001           0

In [34]:
cb_ticker_list = norm_products['id'].tolist()
req_ticker_list = ['BTC-USD', 'ETH-USD', 'DOGE-USD', 'SHIB-USD', 'XRP-USD', 'LINK-USD', 'MKR-USD', 'ADA-USD']
req_ticker_availability = {}
for tkr in req_ticker_list:
    req_ticker_availability[tkr] = tkr in cb_ticker_list

In [35]:
print(req_ticker_availability)

{'BTC-USD': True, 'ETH-USD': True, 'DOGE-USD': True, 'SHIB-USD': True, 'XRP-USD': False, 'LINK-USD': True, 'MKR-USD': True, 'ADA-USD': True}


In [36]:
temp_file_handle = open("temp_coinbase_ticker_list.txt", 'w')
for tkr in cb_ticker_list:
    temp_file_handle.write(tkr + '\n')
temp_file_handle.close()

In [57]:
max_size = 301
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
temp_granularity = "1m"
temp_interval = timedelta(minutes=(max_size-1)*binsizes[temp_granularity])
temp_ticker = "BTC-USD"
time_format = "%Y-%m-%d %H:%M:%S"
temp_start_string = "2020-12-10 12:10:08"
#temp_end_string = "2020-12-10 13:10:08"
temp_start = datetime.strptime(temp_start_string, time_format)
#temp_end = datetime.strptime(temp_end_string, time_format)
temp_end = temp_start + temp_interval
print(temp_start, temp_end)

2020-12-10 12:10:08 2020-12-10 17:10:08


In [58]:
temp_response = requests.get("https://api.pro.coinbase.com/products/{0}/candles?start={1}&end={2}&granularity={3}"\
                            .format(temp_ticker, temp_start, temp_end, temp_granularity))

In [60]:
temp_norm_response = pd.DataFrame(json.loads(temp_response.text))
temp_norm_response[0] = pd.to_datetime(temp_norm_response[0], unit='s')
print(temp_norm_response[0].tolist())

[Timestamp('2020-12-10 17:10:00'), Timestamp('2020-12-10 17:09:00'), Timestamp('2020-12-10 17:08:00'), Timestamp('2020-12-10 17:07:00'), Timestamp('2020-12-10 17:06:00'), Timestamp('2020-12-10 17:05:00'), Timestamp('2020-12-10 17:04:00'), Timestamp('2020-12-10 17:03:00'), Timestamp('2020-12-10 17:02:00'), Timestamp('2020-12-10 17:01:00'), Timestamp('2020-12-10 17:00:00'), Timestamp('2020-12-10 16:59:00'), Timestamp('2020-12-10 16:58:00'), Timestamp('2020-12-10 16:57:00'), Timestamp('2020-12-10 16:56:00'), Timestamp('2020-12-10 16:55:00'), Timestamp('2020-12-10 16:54:00'), Timestamp('2020-12-10 16:53:00'), Timestamp('2020-12-10 16:52:00'), Timestamp('2020-12-10 16:51:00'), Timestamp('2020-12-10 16:50:00'), Timestamp('2020-12-10 16:49:00'), Timestamp('2020-12-10 16:48:00'), Timestamp('2020-12-10 16:47:00'), Timestamp('2020-12-10 16:46:00'), Timestamp('2020-12-10 16:45:00'), Timestamp('2020-12-10 16:44:00'), Timestamp('2020-12-10 16:43:00'), Timestamp('2020-12-10 16:42:00'), Timestamp('20